In [18]:
from datetime import datetime
import json
import re
import tabulate
import textwrap
import requests

secrets_file = 'secrets.txt'
with open(secrets_file) as file:
    api_key = file.read()

timestamp = datetime.today()
url = 'https://api.nasa.gov/planetary/apod'
params = {'api_key': api_key,
         'count': 5}
r = requests.get(url, params=params)

In [19]:
if r.status_code == 200:
    data = r.json()
    # Filter for images only
    filtered = [image for image in data if image.get('media_type') == 'image']
    # Sort by date
    filtered = sorted(filtered, key= lambda image: image['date'])
    # Arbitrarily use the latest image
    latest = filtered[-1]
    data_dict = dict(title = latest.get('title'),
                     copyright = latest.get('copyright', 'N/A').strip(),
                     url = latest.get('url'),
                     is_hd = False,
                     description = latest.get('explanation'))
    if latest.get('hdurl'):
        data_dict['url'] = latest.get('hdurl')
        data_dict['is_hd'] = True

In [20]:
text = data_dict.get('description')
beginning = ['<doctype html>','<html>', '<head>', '<title>', "Today's APOD Image from NASA", '</title>',
             '</head>', '<body>']
ending = ['</body>', '</html>']

url = f"<a href=\"{data_dict['url']}\">URL</a>"
table = tabulate.tabulate([
    ['Title: ', data_dict['title']],
    ['Copyright: ', data_dict['copyright']],
    ['URL: ', 'INSERT URL HERE'],
    ['Is HD: ', data_dict['is_hd']],
    ['Description: ', data_dict['description']]],
                        tablefmt='html',
                        colalign=('right', 'left'),
                                  maxcolwidths=[None, 79])

# Change URL to be a link in the HTML
linked_table = re.sub(r'INSERT URL HERE', url, table)
date_string = timestamp.strftime('%Y-%m-%d-%H%M')
file = f'APOD-{date_string}JST.html'
with open(file, 'w') as f:
    f.write('\n'.join(beginning))
    f.write('\n')
    f.write(linked_table)
    f.write('\n')
    f.write('\n'.join(ending))